In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from ld_script import *
pd.set_option('display.max_colwidth', None)

In [2]:
S2801d = pd.read_csv(os.path.join("data/subject_tables/unzipped_files/ACSST5Y2019.S2801_2021-01-12T121135/ACSST5Y2019.S2801_data_with_overlays_2021-01-12T121130.csv"), low_memory=False)
S2801m = pd.read_csv(os.path.join("data/subject_tables/unzipped_files/ACSST5Y2019.S2801_2021-01-12T121135/ACSST5Y2019.S2801_metadata_2021-01-12T121130.csv"))

In [3]:
print(f'Shape of data table proper is {S2801d.shape} \n'
    + f'Shape of metadata table is {S2801m.shape}')

Shape of data table proper is (5266, 126) 
Shape of metadata table is (125, 2)


In [4]:
S2801d.head(2)

,GEO_ID,NAME,S2801_C01_001E,S2801_C01_001M,S2801_C01_002E,S2801_C01_002M,S2801_C01_003E,S2801_C01_003M,S2801_C01_004E,S2801_C01_004M,...,S2801_C02_027E,S2801_C02_027M,S2801_C02_028E,S2801_C02_028M,S2801_C02_029E,S2801_C02_029M,S2801_C02_030E,S2801_C02_030M,S2801_C02_031E,S2801_C02_031M
0,id,Geographic Area Name,Estimate!!Total!!Total households,Margin of Error!!Total!!Total households,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:,Margin of Error!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Desktop or laptop,Margin of Error!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Desktop or laptop,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Desktop or laptop!!Desktop or laptop with no other type of computing device,Margin of Error!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Desktop or laptop!!Desktop or laptop with no other type of computing device,...,"Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $74,999:!!Without an Internet subscription","Margin of Error!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $74,999:!!Without an Internet subscription","Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:","Margin of Error!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:","Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!With dial-up Internet subscription alone","Margin of Error!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!With dial-up Internet subscription alone","Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!With a broadband Internet subscription","Margin of Error!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!With a broadband Internet subscription","Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!Without an Internet subscription","Margin of Error!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!Without an Internet subscription"
1,1400000US48001950100,"Census Tract 9501, Anderson County, Texas",1790,173,1436,171,1256,147,61,29,...,29.6,12.3,(X),(X),0.0,6.1,88.7,6.6,11.3,6.6


In [5]:
S2801m.head(2)

,GEO_ID,id
0,NAME,Geographic Area Name
1,S2801_C01_001E,Estimate!!Total!!Total households


In [6]:
# Custom function defined in ld_scripts.py in project folder to break apart GEO_ID column appropriately
S2801d = parse_geo_id(S2801d)

In [8]:
S2801d, S2801m = drop_and_normalize(S2801d, S2801m)

In [11]:
index_locations = [3, *(list(range(11,15,2)))]
index_locations = [x-3 for x in index_locations]                                                                                                     

In [12]:
S2801m.iloc[index_locations]

,GEO_ID,id
0,S2801_C01_001E,Estimate!!Total!!Total households
8,S2801_C01_005E,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Smartphone
10,S2801_C01_006E,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Smartphone!!Smartphone with no other type of computing device


In [13]:
wc = create_wanted_columns(S2801d, S2801m, index_locations)

In [14]:
S2801d = S2801d[wc].copy()

In [21]:
S2801d.head()

,S2801_C01_001E,S2801_C01_005E,S2801_C01_006E,FIPS,STATEFP,COUNTYFP,TRACTCE
0,Estimate!!Total!!Total households,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Smartphone,Estimate!!Total!!Total households!!TYPES OF COMPUTER!!Has one or more types of computing devices:!!Smartphone!!Smartphone with no other type of computing device,_fips,_statefp,_countyfp,_tractce
1,1790,1341,89,48001950100,48,001,950100
2,95,89,5,48001950401,48,001,950401
3,112,93,0,48001950402,48,001,950402
4,1584,1180,203,48001950500,48,001,950500


In [22]:
meta_dict = dict(zip(S2801d.columns, S2801d.iloc[0,:]))

In [23]:
reform_dict(meta_dict)

In [24]:
dict_to_metadata(meta_dict, "S2801_metadata.csv")

Successfully created metadata table at data/processed_data/metadata/S2801_metadata.csv


In [25]:
S2801d.drop(index=0, axis=0, inplace=True)
S2801d.reset_index(drop=True, inplace=True)

In [27]:
pd_totals_path = 'data/processed_data/totals'
with open(os.path.join(pd_totals_path, 'S2801_totals.csv'), 'w', newline='') as outputfile:
    S2801d.to_csv(outputfile, header=True, index=False) 